In [3]:
# Verify
print("Hello Slicer")
# Setup
import numpy as np
import JupyterNotebooksLib as slicernb
import logging
import time
import sys
import pickle
from collections import deque
from queue import PriorityQueue

from importlib import reload

reload(logging)
# create logger
logger = logging.getLogger('scratch_logger')
logger.setLevel(logging.DEBUG)

logstream_ch = logging.StreamHandler()
logstream_ch.setLevel(logging.INFO)

logfile_ch = logging.FileHandler('bfs.log')
logfile_ch.setLevel(logging.DEBUG)

logger.addHandler(logstream_ch)
logger.addHandler(logfile_ch)

loadScene("data/slicer_gs_clean_update1/2023-10-26-Scene.mrml")

def multi_add_fiducial(fid_node, nodes_ras_Nx3):
    for i in range(len(nodes_ras_Nx3)):
        fid_node.AddFiducialFromArray(nodes_ras_Nx3[i,:])
        
def coord2z(i,j,k):
    return i * (262144) + j * (512) + k
def z2coord(z):
    # a[i//(6*7),(i%(6*7)//7),i%(7)]
    return z//(512*512), z%(262144)//512, z%512
def vtk4x4matrix_to_numpy(matrix):
    """
    Copies the elements of a vtkMatrix4x4 into a numpy array.

    :param matrix: The matrix to be copied into an array.
    :type matrix: vtk.vtkMatrix4x4
    :rtype: numpy.ndarray
    """
    m = np.ones((4, 4))
    for i in range(4):
        for j in range(4):
            m[i, j] = matrix.GetElement(i, j)
    return m
## Global
VOLUME_NODE_NAME = "14: Body Soft Tissue"
OUT_A_NODE_NAME = "out_a"
OUT_B_NODE_NAME = "out_b"
OUT_C_NODE_NAME = "out_c"
MYNULL = -2_147_483_648

volume_node = getNode(VOLUME_NODE_NAME)
volume_display = volume_node.GetDisplayNode()
volume_display.InterpolateOff()

outa_node = getNode(OUT_A_NODE_NAME)
outa_node.GetDisplayNode().InterpolateOff()

outb_node = getNode(OUT_B_NODE_NAME)
outb_node.GetDisplayNode().InterpolateOff()

outc_node = getNode(OUT_C_NODE_NAME)
outc_node.GetDisplayNode().InterpolateOff()

lad_node = getFirstNodeByName("LAD")
lcx_node = getFirstNodeByName("LCX")
rca_node = getFirstNodeByName("RCA")

v_arr = slicer.util.array(VOLUME_NODE_NAME)
print(v_arr.shape)
print(v_arr.min())
print(v_arr.max())
print(v_arr.dtype)

x = vtk.vtkMatrix4x4()
volume_node.GetIJKToRASMatrix(x)
ijk2ras_mat = vtk4x4matrix_to_numpy(x)

volume_node.GetRASToIJKMatrix(x)
ras2ijk_mat = vtk4x4matrix_to_numpy(x)

# ras2ijk_mat @ np.array([326,192,160,1]).reshape(-1,1)

Hello Slicer
(275, 512, 512)
-3024
1960
int32


In [6]:
def kji_list2ras(kji_tuples):
    return kji2ras(np.array(kji_tuples).T)

def kji2ras(kji_3xN):
    return (ijk2ras_mat @ np.pad(kji_3xN[::-1,:], ((0,1), (0,0)), constant_values=1))[:3,:]

# x2 = kji_list2ras(full_path[:2])
# np.linalg.norm(x2[:,0] - x2[:,1]) * weight_arr[0,0,0]

In [12]:
ijk_spacing = volume_node.GetSpacing()

In [9]:
origin_vec = kji_list2ras([(0,0,0)])
origin_vec

array([[ 222.9],
       [ 200. ],
       [-589. ]])

In [10]:
origin_vec = kji_list2ras([(0,0,0)])[:,0]
for k in range(-1,2):
    for j in range(-1,2):
        for i in range(-1,2):
            vec = kji_list2ras([(k,j,i)])[:,0]
            print(np.linalg.norm(origin_vec - vec))

1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.1048543456039805
0.78125
1.1048543456039805
0.78125
0.0
0.78125
1.1048543456039805
0.78125
1.1048543456039805
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923


In [14]:
origin_vec = kji_list2ras([(0,0,0)])[:,0]
for k in range(-1,2):
    for j in range(-1,2):
        for i in range(-1,2):
            print(((i*ijk_spacing[0])**2 + (j*ijk_spacing[1])**2 + (k*ijk_spacing[2])**2)**0.5)

1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.1048543456039805
0.78125
1.1048543456039805
0.78125
0.0
0.78125
1.1048543456039805
0.78125
1.1048543456039805
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923
